In [1]:
from pathlib import Path

from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer

/home/paul/.conda/envs/codetf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/paul/.conda/envs/codetf/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /home/paul/.conda/envs/tf/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/paul/.conda/envs/codetf/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/paul/.conda/envs/codetf/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/paul/.conda/envs/codetf did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


In [2]:
root_dir = Path.cwd().parent

In [3]:
datasets = load_dataset("json", data_files={
    "train": str(root_dir / "data" / "train.jsonl")
}, cache_dir=root_dir / "data" / "cache", split="train")

Found cached dataset json (/home/paul/projects/edu/master/mdl-ii/src/data/cache/json/default-99fd0a96c2329cb4/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [4]:
# datasets = load_dataset("code_x_glue_ct_code_to_text", 'python', split="train")
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5p-220m")

def preprocess_function(examples):
    source = [' '.join(ex) for ex in examples["code_tokens"]]
    target = [' '.join(ex) for ex in examples["docstring_tokens"]]

    model_inputs = tokenizer(source, max_length=320, padding="max_length", truncation=True)
    labels = tokenizer(target, max_length=128, padding="max_length", truncation=True)

    model_inputs["labels"] = labels["input_ids"].copy()
    model_inputs["labels"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in model_inputs["labels"]
    ]
    return model_inputs

train_data = datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=datasets.column_names,
    num_proc=64,
    load_from_cache_file=False,
)

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5p-220m-py")
model.to("cuda")

training_args = TrainingArguments(
    output_dir=root_dir / "modeling" / "models" / "codet5p_220m_py",
    overwrite_output_dir=False,

    do_train=True,
    save_strategy='epoch',

    num_train_epochs=2,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,

    learning_rate=5e-5,
    weight_decay=0.05,
    warmup_steps=200,

    logging_dir=root_dir / "modeling" / "models" / "codet5p_220m_py",
    logging_first_step=True,
    logging_steps=100,
    save_total_limit=1,

    dataloader_drop_last=True,
    dataloader_num_workers=4,

    local_rank=-1,
    deepspeed=None,
    fp16=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
)

In [8]:
trainer.train()

Step,Training Loss
1,4.905000


KeyboardInterrupt: 

In [ ]:
trainer.save_model(root_dir / "modeling" / "models" / "codet5p_220m_py")